In [ ]:
!pip install loguru deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
# @title ## 1. Setup: Mount Google Drive & Install Libraries
# Purpose: This cell connects your Colab environment to your Google Drive
# and installs the required 'transformers' library for BERT.

from google.colab import drive
import os

# Mount your Google Drive. You'll be prompted to authorize this.
drive.mount('/content/drive')

# Install the Hugging Face transformers library quietly
!pip install -q transformers

print("✅ Google Drive mounted and libraries installed.")


# @title ## 2. Clone Your GitHub Repository
# Purpose: This cell downloads your code and data from GitHub into the Colab environment.

# Define the repository URL
repo_url = "https://github.com/nhahub/NHA-112.git"
repo_name = "NHA-112"

# Clone the repository
if not os.path.exists(repo_name):
    !git clone {repo_url}
else:
    print("Repository already cloned.")

# Change the working directory into your repository
os.chdir(repo_name)
print(f"✅ Repository cloned. Current working directory: {os.getcwd()}")



Mounted at /content/drive
✅ Google Drive mounted and libraries installed.
Cloning into 'NHA-112'...
remote: Enumerating objects: 815, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 815 (delta 2), reused 2 (delta 2), pack-reused 808 (from 1)
Receiving objects: 100% (815/815), 98.38 MiB | 31.06 MiB/s, done.
Resolving deltas: 100% (154/154), done.
✅ Repository cloned. Current working directory: /content/NHA-112


In [ ]:
# @title ## 3. Import Your BertTextClassifier Class
# Purpose: This cell adds your repository's code to Python's path and imports
# the necessary libraries and your custom classifier.
import sys
import pandas as pd
from sklearn.model_selection import train_test_split

# Add the current directory to the Python path to find your module
sys.path.append('.')

# Now you can import your class
from dataEngineer.modeling.Deeplearning2 import *
from dataEngineer.pipeLine import *

print("✅ SentimentAnalysisModel class imported successfully.")

✅ SentimentAnalysisModel class imported successfully.


In [ ]:
data_path = "/content/NHA-112/data/interim/reddit_complaints_dataset.csv"

In [ ]:
df = pd.read_csv(data_path)

In [ ]:
df['text'] = df['text'].convert_dtypes('object')
df = df[df['text'].str.strip() != ""]
df.reset_index(drop=True, inplace=True)
cleaning_pipeline = Pipeline([("text_preprocessor", NltkTextPreprocessor())])

print("Applying sklearn pipeline for text cleaning and lemmatization...")
processed_text_series = pd.Series(
    cleaning_pipeline.fit_transform(df['text']), name="processed_text"
)
df["processed_text"] = processed_text_series

Applying sklearn pipeline for text cleaning and lemmatization...


In [ ]:
df.columns

Index(['category', 'subreddit', 'problem_type', 'title', 'text',
       'processed_text'],
      dtype='object')

In [ ]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# ---------------------------------------------------------
# ✨ FUNCTION: Evaluate a model using your own class handler
# ---------------------------------------------------------
def evaluate_model(model_path, model_name, df, text_column, category_column, subcategory_column, batch_size=8):
    print(f"\n📌 Evaluating model: {model_path}")

    handler = MultiOutputClassificationModel(model_name=model_name, model_path=model_path)

    # Load model
    model, optimizer = handler.load()
    if model is None:
        print(f"❌ Cannot load model: {model_path}")
        return None

    model.eval()

    # Encode labels (same process used in training)
    encoded_cat = handler.label_encoder_category.transform(df[category_column])
    encoded_sub = handler.label_encoder_subcategory.transform(df[subcategory_column])

    texts = df[text_column].values

    # Train/Val split EXACT like training
    train_texts, val_texts, train_cat, val_cat, train_sub, val_sub = train_test_split(
        texts, encoded_cat, encoded_sub, test_size=0.2, random_state=42
    )

    # Build dataset + dataloader
    val_dataset = MultiOutputDataset(val_texts, val_cat, val_sub, handler.tokenizer)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    criterion = torch.nn.CrossEntropyLoss()
    total_loss = 0
    all_preds_cat = []
    all_preds_sub = []
    all_labels_cat = []
    all_labels_sub = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(handler.device)
            attention_mask = batch['attention_mask'].to(handler.device)
            labels_cat = batch['labels_category'].to(handler.device)
            labels_sub = batch['labels_subcategory'].to(handler.device)

            out_cat, out_sub = model(input_ids, attention_mask)

            # Total loss
            loss_cat = criterion(out_cat, labels_cat)
            loss_sub = criterion(out_sub, labels_sub)
            total_loss += (loss_cat + loss_sub).item()

            _, pred_cat = torch.max(out_cat, 1)
            _, pred_sub = torch.max(out_sub, 1)

            all_preds_cat.extend(pred_cat.cpu().numpy())
            all_preds_sub.extend(pred_sub.cpu().numpy())
            all_labels_cat.extend(labels_cat.cpu().numpy())
            all_labels_sub.extend(labels_sub.cpu().numpy())

    acc_cat = accuracy_score(all_labels_cat, all_preds_cat)
    acc_sub = accuracy_score(all_labels_sub, all_preds_sub)

    results = {
        "model_path": model_path,
        "val_loss": total_loss / len(val_loader),
        "category_accuracy": acc_cat,
        "subcategory_accuracy": acc_sub,
        "avg_accuracy": (acc_cat + acc_sub) / 2
    }

    return results

# ---------------------------------------------------------
# ✨ COMPARE MULTIPLE MODELS
# ---------------------------------------------------------
model_paths = [
    "/content/drive/MyDrive/DataEngineerProject_productSentementAnaylsis/bert_multiclassification_uncleanedData/sentiment_classifier.pth",
    "/content/drive/MyDrive/DataEngineerProject_productSentementAnaylsis/text_classifier_cleanedData/sentiment_classifier.pth",
    "/content/drive/MyDrive/DataEngineerProject_productSentementAnaylsis/text_classifier_cleanedData_50epoch_2e-6/sentiment_classifier.pth",
]

results = []
for mp in model_paths:
    res = evaluate_model(
        model_path=mp,
        model_name="distilbert-base-uncased",
        df=df,
        text_column='processed_text',
        category_column='category',
        subcategory_column='problem_type'
    )
    if res:
        results.append(res)

# ---------------------------------------------------------
# ✨ PRINT RESULTS SORTED BY BEST MODEL
# ---------------------------------------------------------
if results:
    print("\n\n===================== MODEL COMPARISON =====================")
    results = sorted(results, key=lambda x: x['avg_accuracy'], reverse=True)

    for r in results:
        print(f"\nModel: {r['model_path']}")
        print(f" - Validation Loss:        {r['val_loss']:.4f}")
        print(f" - Category Accuracy:      {r['category_accuracy']*100:.2f}%")
        print(f" - Sub-category Accuracy:  {r['subcategory_accuracy']*100:.2f}%")
        print(f" - Overall Avg Accuracy:   {(r['avg_accuracy']*100):.2f}%")

    print("\n🏆 BEST MODEL:")
    print(results[0])
else:
    print("❌ No models could be evaluated.")



📌 Evaluating model: /content/drive/MyDrive/DataEngineerProject_productSentementAnaylsis/bert_multiclassification_uncleanedData/sentiment_classifier.pth
Using device: cuda
✅ Loaded model from /content/drive/MyDrive/DataEngineerProject_productSentementAnaylsis/bert_multiclassification_uncleanedData/sentiment_classifier.pth.

📌 Evaluating model: /content/drive/MyDrive/DataEngineerProject_productSentementAnaylsis/text_classifier_cleanedData/sentiment_classifier.pth
Using device: cuda
✅ Loaded model from /content/drive/MyDrive/DataEngineerProject_productSentementAnaylsis/text_classifier_cleanedData/sentiment_classifier.pth.

📌 Evaluating model: /content/drive/MyDrive/DataEngineerProject_productSentementAnaylsis/text_classifier_cleanedData_50epoch_2e-6/sentiment_classifier.pth
Using device: cuda
✅ Loaded model from /content/drive/MyDrive/DataEngineerProject_productSentementAnaylsis/text_classifier_cleanedData_50epoch_2e-6/sentiment_classifier.pth.


===================== MODEL COMPARISON ==